In [9]:
from rich import print
from sqlalchemy import create_engine, ForeignKey, String, Text, Integer
from sqlalchemy.orm import (
    Session,
    DeclarativeBase,
    Mapped,
    mapped_column,
    relationship,
)

In [10]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [11]:
class Base(DeclarativeBase):
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)

In [12]:
# Связь Один ко многим
class Users(Base):
    __tablename__ = 'users'
    name: Mapped[str] = mapped_column(String(20), unique=True)
    fullname: Mapped[str] = mapped_column(String(100), server_default='')
    posts: Mapped[list['Posts']] = relationship(back_populates='author')

    def __repr__(self):
        return f'User(name={self.name}, fullname={self.fullname})'


class Posts(Base):
    __tablename__ = 'posts'
    author_id: Mapped[int] = mapped_column(ForeignKey('users.id'))
    title: Mapped[str] = mapped_column(String(150), unique=True)
    content: Mapped[str] = mapped_column(Text)
    author: Mapped['Users'] = relationship(back_populates='posts')

    def _repr__(self):
        return f'Post(title={self.title}, content={self.content[:30]})'
    
    def __str__(self) -> str:
        return f'Post(title={self.title}, content={self.content[:30]})'

In [13]:
Base.metadata.create_all(engine)

2025-09-08 13:55:30,889 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-08 13:55:30,889 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-09-08 13:55:30,890 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:55:30,890 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-09-08 13:55:30,891 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:55:30,891 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("posts")
2025-09-08 13:55:30,891 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:55:30,892 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("posts")
2025-09-08 13:55:30,892 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:55:30,893 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	name VARCHAR(20) NOT NULL, 
	fullname VARCHAR(100) DEFAULT '' NOT NULL, 
	id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2025-09-08 13:55:30,893 INFO sqlalchemy.engine.Engine [no key 0.00027s] ()
2025-09-08 13:55:30,894 INFO s

In [14]:
# Включаем поддержку Foreign keys в SQLite
from sqlalchemy import event

def _fk_pragma_on_connect(dbapi_con, con_record):
    dbapi_con.execute('pragma foreign_keys=ON')

event.listen(engine, 'connect', _fk_pragma_on_connect)

In [15]:
with Session(engine) as session:
    users = [
        Users(name='ivan', fullname='Ivan Ivanov'),
        Users(name='petr', fullname='Petr Petrov'),
        Users(name='max', fullname='Max Maximov'),
    ]

    session.add_all(users)
    session.commit()

2025-09-08 13:55:30,929 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-08 13:55:30,931 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?) RETURNING id
2025-09-08 13:55:30,931 INFO sqlalchemy.engine.Engine [generated in 0.00005s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('ivan', 'Ivan Ivanov')
2025-09-08 13:55:30,932 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?) RETURNING id
2025-09-08 13:55:30,932 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('petr', 'Petr Petrov')
2025-09-08 13:55:30,932 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?) RETURNING id
2025-09-08 13:55:30,933 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('max', 'Max Maximov')
2025-09-08 13:55:30,934 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
with Session(engine) as session:
    posts = [
        Posts(title='Post 1', content='Content 1', author=users[0]),
        Posts(title='Post 2', content='Content 2', author=users[0]),
        Posts(title='Post 3', content='Content 3', author=users[1]),
        Posts(title='Post 4', content='Content 4', author=users[2]),
    ]

    session.add_all(posts)
    session.commit()

2025-09-08 13:55:30,948 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-08 13:55:30,949 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname, users.id AS users_id 
FROM users 
WHERE users.id = ?
2025-09-08 13:55:30,950 INFO sqlalchemy.engine.Engine [generated in 0.00050s] (1,)
2025-09-08 13:55:30,950 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname, users.id AS users_id 
FROM users 
WHERE users.id = ?
2025-09-08 13:55:30,951 INFO sqlalchemy.engine.Engine [cached since 0.001774s ago] (2,)
2025-09-08 13:55:30,951 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname, users.id AS users_id 
FROM users 
WHERE users.id = ?
2025-09-08 13:55:30,952 INFO sqlalchemy.engine.Engine [cached since 0.00269s ago] (3,)
2025-09-08 13:55:30,953 INFO sqlalchemy.engine.Engine INSERT INTO posts (author_id, title, content) VALUES (?, ?, ?) RETURNING id


2025-09-08 13:55:30,953 INFO sqlalchemy.engine.Engine [generated in 0.00005s (insertmanyvalues) 1/4 (ordered; batch not supported)] (1, 'Post 1', 'Content 1')
2025-09-08 13:55:30,954 INFO sqlalchemy.engine.Engine INSERT INTO posts (author_id, title, content) VALUES (?, ?, ?) RETURNING id
2025-09-08 13:55:30,955 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/4 (ordered; batch not supported)] (1, 'Post 2', 'Content 2')
2025-09-08 13:55:30,955 INFO sqlalchemy.engine.Engine INSERT INTO posts (author_id, title, content) VALUES (?, ?, ?) RETURNING id
2025-09-08 13:55:30,956 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/4 (ordered; batch not supported)] (2, 'Post 3', 'Content 3')
2025-09-08 13:55:30,957 INFO sqlalchemy.engine.Engine INSERT INTO posts (author_id, title, content) VALUES (?, ?, ?) RETURNING id
2025-09-08 13:55:30,957 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/4 (ordered; batch not supported)] (3, 'Post 4', 'Content 4')
2025-09-08 13:55:30,958 INFO sqlalchemy.engine

In [22]:
with Session(engine) as session:
    user_ivan = session.query(Users).filter(Users.name == 'ivan').first()

    print(user_ivan)

    for post in user_ivan.posts:
        print(f'Post: {post}\nUser: {post.author}')

2025-09-08 13:57:16,212 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-08 13:57:16,213 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname, users.id AS users_id 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2025-09-08 13:57:16,213 INFO sqlalchemy.engine.Engine [cached since 105.2s ago] ('ivan', 1, 0)


User(name=ivan, fullname=Ivan Ivanov)

2025-09-08 13:57:16,215 INFO sqlalchemy.engine.Engine SELECT posts.author_id AS posts_author_id, posts.title AS posts_title, posts.content AS posts_content, posts.id AS posts_id 
FROM posts 
WHERE ? = posts.author_id
2025-09-08 13:57:16,216 INFO sqlalchemy.engine.Engine [cached since 105.2s ago] (1,)


Post: Post(title=Post 1, content=Content 1)
User: User(name=ivan, fullname=Ivan Ivanov)

Post: Post(title=Post 2, content=Content 2)
User: User(name=ivan, fullname=Ivan Ivanov)

2025-09-08 13:57:16,218 INFO sqlalchemy.engine.Engine ROLLBACK
